# Random Forest
Cold nuclei from botton quintile.  
Run on Windows / Precision.
Train on 5-class cancer classification.  
Input is the nucleus+RBC rollup, training set, cold nuclei only.    
Run 5-fold cross validation.  
Print the mean accuracy. 

The Image.csv is a copy of the original.  
The RBC_rollup.csv is a copy of the original.  
The Nucleus_rollup.csv summarizes only the low-temperature nuclei.

In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)
#import tensorflow as tf
#tf.config.list_physical_devices('GPU')

2022-06-16 17:14:20.263318
Python 3.9.6
sklearn 1.1.1


In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.feature_selection import RFE
import joblib # used to dump/load sklearn models
from CellProfiler_Util import CP_Util
from RandomForestUtil import RF_Util

In [3]:
BASE_DIR='D:\\Adjeroh\\Naved\\CP_80K_ColdQuint\\'  # append Output0/ etc
CLASSES=range(0,6)  # use all 6 classes
CLASSES=range(1,6)  # use just 5 classes
MODELS_DIR='D:\\Adjeroh\\Naved\\CP_80K\\models\\RandomForest.24'

In [4]:
print(datetime.datetime.now())
PREFIX='abc_'
def set_prefix(prefix):
    global PREFIX
    PREFIX=prefix
def get_prefix(col_name):
    global PREFIX
    return PREFIX+col_name  # hard coded for now
def load_all_classes():
    X = None
    y = None
    for i in CLASSES:
        print(datetime.datetime.now())
        print("Process class",i)
        FULL_PATH=BASE_DIR+'Output'+str(i)+'/'
        cp = CP_Util(FULL_PATH)
        cp.train_test_split()
        Xnuc = cp.get_nucleus_rollup()
        set_prefix('Nuc_')
        Xnuc.rename(get_prefix,axis='columns',inplace=True)
        #Xrbc = cp.get_RBC_rollup()
        #set_prefix('Rbc_')
        #Xrbc.rename(get_prefix,axis='columns',inplace=True)
        #Xi = Xnuc.join(Xrbc,how='outer')
        Xi = Xnuc
        size = len(Xi)
        yi = np.ones(size) * i   # e.g. class 3
        if X is None:
            X = Xi
            y = yi
        else:
            X = pd.concat( (X,Xi) )
            y = np.concatenate( (y,yi) )
    X.fillna(0,inplace=True)  
    return X,y
Xtrain,ytrain=load_all_classes()
Xtrain

2022-06-16 17:14:21.555927
2022-06-16 17:14:21.555927
Process class 1
Train: 17 participants, 28 WSI, 10925 patches.
Test: 4 participants, 5 WSI, 1977 patches.
2022-06-16 17:14:39.202736
Process class 2
Train: 14 participants, 30 WSI, 11138 patches.
Test: 3 participants, 7 WSI, 2812 patches.
2022-06-16 17:14:58.002926
Process class 3
Train: 11 participants, 13 WSI, 5006 patches.
Test: 3 participants, 4 WSI, 1358 patches.
2022-06-16 17:15:06.962266
Process class 4
Train: 6 participants, 7 WSI, 2809 patches.
Test: 1 participants, 1 WSI, 388 patches.
2022-06-16 17:15:11.794676
Process class 5
Train: 2 participants, 3 WSI, 1195 patches.
Test: 1 participants, 1 WSI, 396 patches.


,Nuc_AreaShape_Area_count,Nuc_AreaShape_Area_mean,Nuc_AreaShape_Area_std,Nuc_AreaShape_Area_min,Nuc_AreaShape_Area_25%,Nuc_AreaShape_Area_50%,Nuc_AreaShape_Area_75%,Nuc_AreaShape_Area_max,Nuc_AreaShape_BoundingBoxArea_count,Nuc_AreaShape_BoundingBoxArea_mean,...,Nuc_Texture_Variance_Hematoxylin_7_02_256_75%,Nuc_Texture_Variance_Hematoxylin_7_02_256_max,Nuc_Texture_Variance_Hematoxylin_7_03_256_count,Nuc_Texture_Variance_Hematoxylin_7_03_256_mean,Nuc_Texture_Variance_Hematoxylin_7_03_256_std,Nuc_Texture_Variance_Hematoxylin_7_03_256_min,Nuc_Texture_Variance_Hematoxylin_7_03_256_25%,Nuc_Texture_Variance_Hematoxylin_7_03_256_50%,Nuc_Texture_Variance_Hematoxylin_7_03_256_75%,Nuc_Texture_Variance_Hematoxylin_7_03_256_max
PatchNumber,,,,,,,,,,,,,,,,,,,,,
406,10.0,421.700000,152.205307,197.0,326.00,394.5,480.25,729.0,10.0,615.300000,...,792.022735,933.042532,10.0,634.901286,256.294466,260.869336,498.726894,595.638929,769.018072,1039.626797
407,10.0,435.000000,218.927182,196.0,344.25,385.5,441.50,964.0,10.0,646.300000,...,1171.988115,2192.973417,10.0,958.974846,609.157133,0.000000,621.713766,889.654657,1219.561407,2200.376059
408,8.0,329.125000,84.226629,223.0,256.00,329.5,384.00,461.0,8.0,446.750000,...,1683.335764,2508.869890,8.0,1537.624889,753.751098,949.343388,1135.274558,1220.133255,1620.556098,3262.232461
409,7.0,370.428571,104.337365,229.0,305.00,369.0,447.50,490.0,7.0,535.285714,...,847.090933,1256.425532,7.0,633.451122,380.035919,118.109912,380.672146,545.261875,925.984585,1157.472607
411,12.0,382.833333,119.589323,203.0,321.25,342.5,446.50,615.0,12.0,525.916667,...,853.492553,1615.331586,12.0,788.219653,448.216895,227.521415,494.611695,643.855104,892.901094,1608.793600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,1.0,384.000000,0.000000,384.0,384.00,384.0,384.00,384.0,1.0,570.000000,...,3051.999150,3051.999150,1.0,3252.028347,0.000000,3252.028347,3252.028347,3252.028347,3252.028347,3252.028347
1588,5.0,362.800000,156.520606,188.0,207.00,439.0,440.00,540.0,5.0,674.600000,...,2697.422264,3011.793863,5.0,1894.063293,1113.086891,435.222222,1348.354167,1704.394375,2783.133279,3199.212422
1589,5.0,327.400000,99.052007,221.0,286.00,320.0,321.00,489.0,5.0,439.000000,...,1359.689952,2033.936678,5.0,1472.957922,580.871145,795.906227,1002.792011,1638.929748,1679.660152,2247.501471


In [5]:
print(datetime.datetime.now())
print("Shuffle...")
Xtrain,ytrain=shuffle(Xtrain,ytrain)  #set random_state for reproducibility

2022-06-16 17:15:14.505633
Shuffle...


In [6]:
print(datetime.datetime.now())
print('Cross-validation...')
rf1 = RF_Util()
rf1.set_train(Xtrain,ytrain)
cv_scores = rf1.cross_validation()
print(cv_scores)
print('mean %.4f +/- %.4f' % (cv_scores.mean(),cv_scores.std()))

2022-06-16 17:15:14.971142
Cross-validation...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time= 2.2min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.2min remaining:    0.0s


[CV] END .................................................... total time= 2.1min
[CV] END .................................................... total time= 2.1min
[CV] END .................................................... total time= 2.1min
[CV] END .................................................... total time= 2.1min
[0.58036677 0.58540093 0.57928803 0.58737864 0.58000719]
mean 0.5825 +/- 0.0033


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 10.7min finished


In [7]:
print(datetime.datetime.now())
joblib.dump(rf1.get_model(),MODELS_DIR)

2022-06-16 17:25:55.122208


['D:\\Adjeroh\\Naved\\CP_80K\\models\\RandomForest.24']